In [22]:
import tkinter as tk
from tkinter import ttk, Entry, Text, Label, Button, END
import os
import speech_recognition as sr
import gtts
import playsound
from tkinter import messagebox
from googletrans import Translator, LANGUAGES

# Initialize Tkinter window
root = tk.Tk()
root.title("My Translator App")
root.geometry("1100x1100")
root['bg'] = 'lightgrey'

# UI Elements
Label(root, bg="lightgrey", text="LANGUAGE TRANSLATOR", font=("Comic Sans MS", 20, "bold")).pack()
Label(root, bg="lightgrey", text="Enter text to translate", font=("Helvetica", 14)).place(x=50, y=150)
input_text = Entry(root, width=40)
input_text.place(x=250, y=155)

Label(root, bg="lightgrey", text="Translated Text", font=("Helvetica", 16)).place(x=550, y=155)
output = Text(root, font=("Helvetica", 13), height=5, width=40)
output.place(x=700, y=155)


# Language dropdowns
language = list(LANGUAGES.values())
lang = ttk.Combobox(root, values=language, font=("Helvetica", 13), width=47)
lang.place(x=50, y=100)
lang.set("Select your language")

lang2 = ttk.Combobox(root, values=language, font=("Helvetica", 13), width=55)
lang2.place(x=550, y=100)
lang2.set("Select a language to translate into")

# Function to get language code from name
def get_lang_code(lang_name):
    for code, name in LANGUAGES.items():
        if name.lower() == lang_name.lower():
            return code
    return None

# Translation Function
def Translate():
    translator = Translator()
    src_lang_code = get_lang_code(lang.get())  # Get language code
    dest_lang_code = get_lang_code(lang2.get())  # Get destination language code

    if not src_lang_code or not dest_lang_code:
        response = messagebox.askokcancel("Confirm", "Please Select A Language")
        if response:
            print("User clicked OK")
        else:
            print("User clicked Cancel")
        return
        

    translated_text = translator.translate(text=input_text.get(), src=src_lang_code, dest=dest_lang_code)
    output.delete(1.0, END)
    output.insert(END, translated_text.text)

# Speech Recognition & Translation Function
def Audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        voice = recognizer.listen(source)

        # Get selected language from dropdown
        source_lang_code = get_lang_code(lang.get())

        if not source_lang_code:
            response = messagebox.askokcancel("Confirm", "Please Select A Language")
            if response:
                print("User clicked OK")
            else:
                print("User clicked Cancel")
            return

        # Recognize speech in the selected language
        text = recognizer.recognize_google(voice, language=source_lang_code)
        print("You said:", text)

        # Display recognized text in input_text box
        input_text.delete(0, END)
        input_text.insert(0, text)

    # Translate text
    translator = Translator()
    target_lang_code = get_lang_code(lang2.get())

    if not target_lang_code:
        print("Invalid target language selected!")
        return

    translated_text = translator.translate(text=text, dest=target_lang_code)

    # Display translated text in output text box
    output.delete(1.0, END)
    output.insert(END, translated_text.text)

    # Convert translated text to speech
    converted_audio = gtts.gTTS(translated_text.text, lang=target_lang_code)

    # Handle file overwrite issues
    audio_file = "hello.mp3"
    if os.path.exists(audio_file):
        try:
            os.remove(audio_file)  
        except PermissionError:
            print("File in use, retrying...")
            os.rename(audio_file, "temp.mp3")  
            os.remove("temp.mp3")  

    # Save and play audio
    converted_audio.save(audio_file)
    playsound.playsound(audio_file)

# Buttons
Button(root, text='Translate in text', font=("Helvetica", 13), command=Translate, bg='Dark gray').place(x=50, y=250)
Button(root, text="Talk & Translate", font=("Helvetica", 13), command=Audio, bg='Dark gray').place(x=200, y=250)

root.mainloop()


User clicked OK
